In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/project/pro.csv')

In [ ]:
start_mem = df.memory_usage().sum() / 1024**2
def reduce_mem_usage(df):    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                #if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                #    df[col] = df[col].astype(np.float16)
                #el
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        #else:
            #df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB --> {:.2f} MB (Decreased by {:.1f}%)'.format(
        start_mem, end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
df=reduce_mem_usage(df)

Memory usage of dataframe is 34.20 MB --> 22.71 MB (Decreased by 33.6%)


In [ ]:
df=df.dropna()

In [ ]:
data=df.copy()

In [ ]:
x2=data['Drug'].unique()
x3=data['Condition'].unique()
x4=data['Age'].unique()
x5=data['Sex'].unique()
drug_enc = dict(enumerate(x2.flatten(), 1))
condition_enc = dict(enumerate(x3.flatten(), 1))
age_enc=dict(enumerate(x4.flatten(), 1))
sex_enc=dict(enumerate(x5.flatten(), 1))
def age_encode(x):
    for j in age_enc:
        if x==age_enc[j]:
            return j
def condition_encode(x):
    for k in condition_enc:
        if x==condition_enc[k]:
            return k
def drug_encode(x):
    for l in drug_enc:
        if x==drug_enc[l]:
            return l
def sex_encode(x):
    for m in sex_enc:
        if x==sex_enc[m]:
            return m
data['Age']=data['Age'].apply(lambda x:age_encode(x)).astype(str)
data['Condition']=data['Condition'].apply(lambda x:condition_encode(x)).astype(str)
data['Drug']=data['Drug'].apply(lambda x:drug_encode(x)).astype(str)
data['Sex']=data['Sex'].apply(lambda x:sex_encode(x)).astype(str)

In [ ]:
X=data.drop(columns=['Date','Reviews','sentiment','polarity','Unnamed: 0','DrugId','EaseofUse','Satisfaction','Sides','UsefulCount','Effectiveness','emotion'])

In [ ]:
Y = data['Sides']

In [ ]:
X

,Age,Condition,Drug,Sex
0,1,1,1,1
1,2,2,1,2
2,2,3,2,2
3,3,4,2,2
4,4,1,3,1
...,...,...,...,...
280122,2,393,5011,2
280123,4,393,5011,2
280124,2,393,5011,2
280125,4,393,5011,1


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.3,random_state=1000)
DecisionTreePipeline=Pipeline([('decisiontree_classifier',DecisionTreeClassifier(criterion='entropy', max_depth=50, min_samples_split=75))])
knnPipeline=Pipeline([('knn_classifier',KNeighborsClassifier(n_neighbors=2))])
mypipeline=[DecisionTreePipeline,knnPipeline]
accuracy=0.0
classifier=0
pipeline=" "
PipeLineDict={0:'DecisionTree',1:'KNeighbors'}
#fit the pipeline
for mypipe in mypipeline:
    mypipe.fit(X_train,y_train)
for i,model in enumerate(mypipeline):
    print("{} TestAccuracy:{}".format(PipeLineDict[i],model.score(X_test,y_test)))
for i,model in enumerate(mypipeline):
    if model.score(X_test,y_test)>accuracy:
        accuracy=model.score(X_test,y_test)
        pipeline=model
        classifier=i
print('classifier with the best accuracy:{}'.format(PipeLineDict[classifier]))
import pickle
Pkl_Filename = "pipe_model.pkl"  
with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(mypipe, file)
with open(Pkl_Filename, 'rb') as file:  
    pipe_model = pickle.load(file)
print(pipe_model)

DecisionTree TestAccuracy:0.9401622838895707
KNeighbors TestAccuracy:0.9605013881110965
classifier with the best accuracy:KNeighbors
Pipeline(memory=None,
         steps=[('knn_classifier',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='minkowski', metric_params=None,
                                      n_jobs=None, n_neighbors=2, p=2,
                                      weights='uniform'))],
         verbose=False)


In [ ]:

import streamlit as st
import pickle
import pandas as pd
data=pd.read_csv('/content/drive/MyDrive/project/pro.csv')
data=data.dropna()
x2=data['Drug'].unique()
x3=data['Condition'].unique()
x4=data['Age'].unique()
x5=data['Sex'].unique()
drug_enc = dict(enumerate(x2.flatten(), 1))
condition_enc = dict(enumerate(x3.flatten(), 1))
age_enc=dict(enumerate(x4.flatten(), 1))
sex_enc=dict(enumerate(x5.flatten(), 1))
def age_encode(x):
    for j in age_enc:
        if x==age_enc[j]:
            return j
def condition_encode(x):
    for k in condition_enc:
        if x==condition_enc[k]:
            return k
def drug_encode(x):
    for l in drug_enc:
        if x==drug_enc[l]:
            return l
def sex_encode(x):
    for m in sex_enc:
        if x==sex_enc[m]:
            return m
data['Age']=data['Age'].apply(lambda x:age_encode(x)).astype(str)
data['Condition']=data['Condition'].apply(lambda x:condition_encode(x)).astype(str)
data['Drug']=data['Drug'].apply(lambda x:drug_encode(x)).astype(str)
data['Sex']=data['Sex'].apply(lambda x:sex_encode(x)).astype(str)
Pkl_Filename = "pipe_model.pkl" 
with open(Pkl_Filename, 'rb') as file:  
    pipe_model = pickle.load(file)
st.title(' Drug Side effect Prediction')

st.sidebar.header('User Input Parameters')

@st.cache
def prediction(Age,Condition,Drug,Sex):
    age=age_encode(Age)
    sex=sex_encode(Sex)
    condition=condition_encode(Condition)
    drug=drug_encode(Drug)
    predicted=pipe_model.predict([[age,condition,drug,sex]])
    return predicted
def main():       
    # following lines create boxes in which user can enter data required to make prediction 
    Age = st.sidebar.text_input('Enter your age category')
    Sex = st.sidebar.radio('Gender',("Male","Female"))  
    Condition = st.sidebar.text_input('Enter your condition')  
    Drug = st.sidebar.text_input("drug_name")
    result =""
        
    def user_input_features():
        data={'Age': Age,'Condition': Condition,
              'Drug': Drug,
              'Sex' : Sex
              }
        features = pd.DataFrame(data,index = [1])
        return features
    
    df = user_input_features()
    st.subheader('Customer Details')
    st.write(df,width=500,height=2000)
    if st.button("Predict"):
        result = prediction(Age,Condition,Drug,Sex)
        st.success('you are likely to undergo{} '.format(result))
main()

2021-07-17 07:29:53.742 
  command:

    streamlit run /usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py [ARGUMENTS]


In [ ]:
import streamlit as st
import pickle
import pandas as pd
data=pd.read_csv('/content/drive/MyDrive/project/pro.csv')
data=data.dropna()
x2=data['Drug'].unique()
x3=data['Condition'].unique()
x4=data['Age'].unique()
x5=data['Sex'].unique()
drug_enc = dict(enumerate(x2.flatten(), 1))
condition_enc = dict(enumerate(x3.flatten(), 1))
age_enc=dict(enumerate(x4.flatten(), 1))

In [ ]:
def age_encode(x):
    for j in age_enc:
        if x==age_enc[j]:
            return j

In [ ]:
age_encode('25-34')

2

In [ ]:
age_enc

{1: '75 or over',
 2: '25-34',
 3: '45-54',
 4: '55-64',
 5: '19-24',
 6: '35-44',
 7: '13-18',
 8: '65-74',
 9: '7-12',
 10: '0-2',
 11: '3-6'}

In [ ]:
age_enc[2]

'25-34'